In [ ]:
# TODO: requirements.txt
# pip install binreader
# pip install matplotlib
# pip install numpy
# pip install nbconvert

In [ ]:
from core import Measure, MidiNote
from recording import *
from pupil_info import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from recording import *

recording: Recording = Recording("data/asdf.notes")

human_fours: list[list[Measure]] = recording.human_fours()
print([sum(len(measure.notes) for measure in four) for four in human_fours])

In [ ]:

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

dist1: list[int] = [note.note for four in recording.human_fours() for measure in four for note in measure.notes]
dist2: list[int] = [note.note for four in recording.agent_fours() for measure in four for note in measure.notes]

# We can set the number of bins with the *bins* keyword argument.
axs[0].hist(dist1, bins=10)
axs[1].hist(dist2, bins=10)

plt.show()

In [ ]:
all_pupil_info: list[PupilInfo] = get_all_pupil_info()
responses: np.ndarray[int] = get_responses(all_pupil_info)

In [ ]:
# GET THE AVERAGE ANSWER FOR EVERY QUESTION FOR EVERY SONG

for question in range(NUM_QUESTIONS):
    print(f"Question {question}")
    for song in range(NUM_SONGS):
        res = [responses[pupil][song][algorithm][question] for pupil in range(NUM_PUPILS) for algorithm in range(NUM_ALGORITHMS)]
        res = list(filter(lambda x: x != 0, res))
        print(f"{SONG_NAMES[song]}: {np.average(res)}")
    print()

In [ ]:
# GET THE AVERAGE ANSWER PER QUESTION OVER TIME

for question in range(NUM_QUESTIONS):
    print(f"Question {question}")
    for session in range(NUM_SESSIONS):
        res = [all_pupil_info[pupil].sessions[session].performances[performance].responses[question]
               for pupil in range(NUM_PUPILS) for performance in range(NUM_PERFORMANCES)]
        res = list(filter(lambda x: x != 0, res))
        print(f"Session {session}: {np.average(res)}")

In [ ]:
from dataclasses import replace
from local_features import *

def make_notes_absolute(measures: list[Measure]) -> list[Measure]:
    return [Measure([replace(note, time=(note.time + i)) for note in measures[i].notes]) for i in range(len(measures))]

fours = np.split(np.array(make_notes_absolute(recording.measures)), 4)

metrics = [note_std(four) for four in fours]

print(metrics)